# Example of using LightPipelines on Spark OCR
## Install spark-ocr python packge
Need specify path to `spark-ocr-assembly-[version].jar` or `secret`

In [1]:
license = ""
secret = ""
nlp_secret = ""
aws_access_key = ""
aws_secret_key = ""

version = secret.split("-")[0]
spark_ocr_jar_path = "../../target/scala-2.12"

In [2]:
%%bash
if python -c 'import google.colab' &> /dev/null; then
    echo "Run on Google Colab!"
    echo "Install Open JDK"
    apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
    java -version
fi

In [3]:
import sys
import os

if 'google.colab' in sys.modules:
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

## Initialization of spark session

In [4]:
from pyspark.sql import SparkSession
from sparkocr import start

if license:
    os.environ['JSL_OCR_LICENSE'] = license
    
# Pass nlp_secret=nlp_secret, if you don't have a jar
spark = start(secret=secret,
              jar_path=spark_ocr_jar_path, nlp_internal="5.4.0")

Spark version: 3.4.0
Spark NLP version: 5.4.0
Spark NLP for Healthcare version: 5.4.0
Spark OCR version: 5.4.0



In [5]:
from pyspark.ml import PipelineModel
from pyspark.sql.functions import *
from sparkocr.transformers import *

In [6]:
import pkg_resources
pdfs_path = pkg_resources.resource_filename('sparkocr', 'resources/ocr/pdfs/four_pages_clinical_note')


## Define OCR pipeline

In [7]:
# Transform binary to image
pdf_to_image = PdfToImage()
pdf_to_image.setOutputCol("image")

# Run OCR for each region
ocr = ImageToText()
ocr.setInputCol("image")
ocr.setOutputCol("text")
ocr.setConfidenceThreshold(60)

# OCR pipeline
pipeline = PipelineModel(stages=[
    pdf_to_image,
    ocr
])

## Create LightPipeline

In [8]:
from sparkocr.base import LightPipeline

In [9]:
lp = LightPipeline(pipeline)

In [10]:
%%time
lp.fromLocalPath(pdfs_path)

CPU times: user 78.4 ms, sys: 27.1 ms, total: 106 ms
Wall time: 5.13 s


[{'image': ImageOutput(path: 4_pages_clinical_note.pdf, exception: ),
  'exception': [Light Pipeline Exception(message: [ocr_pipeline_exception::]),
   Light Pipeline Exception(message: [ocr_pipeline_exception::])],
  'text': Annotation(image_to_text, 0, 1730,  
  
  Result Information
  
   
  
   
  
   
  
  Status Provider Status
  Final result i Reviewed
  Entry Date
  
   
  
  Component Results
  
  Component Lab
  Surgical Pathology
  
  Patient Nam
  MR#: JCD-O-3
  Specime
  Ca rCtnome unsthelia| NOS
  
  Final Diagnosis ZBIAO/S3
  
  A. Right ureter, biopsy: n
  -Segment of benign ureter, negative for tumor Sitez Lb lad cNer ,wWwall, mw 5
  -Frozen section diagnosis confirmed
  
  Cw. 4
  
   
  
  B. Left ureter, biopsy: B27 IU
  -Segment of benign ureter, negative for tumor [<a)
  -Frozen section diagnosis confirmed
  
  C. Left external and internal lymph nodes, resection:
  ~Benign fibrovascular tissue, negative for tumor
  -No lymph nodes identified
  
  D. Left internal

## Integration with Spark-NLP pipelines

In [11]:
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

sentenceDetector = SentenceDetector()\
  .setInputCols("document")\
  .setOutputCol("sentence")

tokenizer = Tokenizer()\
  .setInputCols("sentence")\
  .setOutputCol("token")

embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
  .setInputCols(["sentence","token"])\
  .setOutputCol("embeddings")

ner = MedicalNerModel.pretrained("ner_jsl", "en","clinical/models")\
  .setInputCols(["sentence","token","embeddings"])\
  .setOutputCol("ner")

converter = NerConverter()\
  .setInputCols("sentence", "token", "ner")\
  .setOutputCol("entities")


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_jsl download started this may take some time.
[OK!]


## Mixed Pipeline

In [23]:
# OCR pipeline
from pyspark.ml import Pipeline, PipelineModel
from sparkocr.base import LightPipeline

pipeline = Pipeline(stages=[
    pdf_to_image,
    ocr,
    documentAssembler,
    sentenceDetector,
    tokenizer,
    embeddings,
    ner,
    converter
])

empty_data = spark.createDataFrame([[b'', ""]]).toDF("content", "path")
pipeline = pipeline.fit(empty_data)
lp = LightPipeline(pipeline)

In [25]:
result = lp.fromLocalPath(pdfs_path)

In [26]:
import pandas as pd
pd.DataFrame.from_records(result)

,entities,image,document,exception,text,pagenum,positions,token,content,ner,embeddings,sentence
0,"[Annotation(chunk, 236, 240, Right, Map(entity...","ImageOutput(path: 4_pages_clinical_note.pdf, e...","Annotation(document, 0, 1729, \n\nResult Info...",[Light Pipeline Exception(message: [ocr_pipeli...,"Annotation(image_to_text, 0, 1730, \n\nResult...",0,"PositionsOutput(mappings: [{'c': ' \n\n', 'x':...","[Annotation(token, 3, 8, Result, Map(sentence ...","source file 4_pages_clinical_note.pdf, 426742 ...","[Annotation(named_entity, 3, 8, O, Map(word ->...","[Annotation(word_embeddings, 3, 8, Result, Map...","[Annotation(document, 3, 377, Result Informati..."
1,"[Annotation(chunk, 48, 52, tumor, Map(entity -...","ImageOutput(path: 4_pages_clinical_note.pdf, e...","Annotation(document, 0, 1481, \n\n-Surgical r...",[Light Pipeline Exception(message: [ocr_pipeli...,"Annotation(image_to_text, 0, 1482, \n\n-Surgi...",1,"PositionsOutput(mappings: [{'c': ' \n\n', 'x':...","[Annotation(token, 3, 3, -, Map(sentence -> 0)...","source file 4_pages_clinical_note.pdf, 426742 ...","[Annotation(named_entity, 3, 3, O, Map(word ->...","[Annotation(word_embeddings, 3, 3, -, Map(isOO...","Annotation(document, 3, 1480, -Surgical resect..."
2,"[Annotation(chunk, 3, 12, CAP cancer, Map(enti...","ImageOutput(path: 4_pages_clinical_note.pdf, e...","Annotation(document, 0, 1349, \n\nCAP cancer ...",[Light Pipeline Exception(message: [ocr_pipeli...,"Annotation(image_to_text, 0, 1350, \n\nCAP ca...",2,"PositionsOutput(mappings: [{'c': ' \n\n', 'x':...","[Annotation(token, 3, 5, CAP, Map(sentence -> ...","source file 4_pages_clinical_note.pdf, 426742 ...","[Annotation(named_entity, 3, 5, B-Oncological,...","[Annotation(word_embeddings, 3, 5, CAP, Map(is...","[Annotation(document, 3, 486, CAP cancer templ..."
3,"[Annotation(chunk, 6, 9, Left, Map(entity -> D...","ImageOutput(path: 4_pages_clinical_note.pdf, e...","Annotation(document, 0, 2027, \n\nL. Left obt...",[Light Pipeline Exception(message: [ocr_pipeli...,"Annotation(image_to_text, 0, 2028, \n\nL. Lef...",3,"PositionsOutput(mappings: [{'c': ' \n\n', 'x':...","[Annotation(token, 3, 3, L, Map(sentence -> 0)...","source file 4_pages_clinical_note.pdf, 426742 ...","[Annotation(named_entity, 3, 3, O, Map(word ->...","[Annotation(word_embeddings, 3, 3, L, Map(isOO...","[Annotation(document, 3, 271, L. Left obturato..."
